In [1]:
import os, sys
# os.system('conda activate open3d')
import open3d as o3d
import numpy as np
import json
from queue import Queue
import pickle
import cv2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
'''
- `mesh.refined.v2.obj`：该文件包含重建的网格模型。
- `mesh.refined.mtl`：该文件是网格模型对应的材质文件。
- `mesh.refined_0.png`：该文件是网格模型的纹理文件。
- `sequence.zip`：这是一个压缩文件，包含了校准的RGB-D传感器流，包括彩色和深度帧，以及相机位姿。
- `labels.instances.annotated.v2.ply`：该文件是网格模型的语义分割可视化文件。
- `mesh.refined.0.010000.segs.v2.json`：该文件包含了注释网格的过分割信息。
- `semseg.v2.json`：该文件包含了网格模型的实例分割信息，包括标签。
'''
print("Load a ply point cloud, print it, and render it")
# pcd = o3d.io.read_point_cloud("scenes/scene0000_00/scene0000_00_vh_clean_2.ply")
pcd = o3d.io.read_triangle_mesh("scenes/scene0000_00/scene0000_00_vh_clean_2.ply")

o3d.visualization.draw_geometries([pcd])

Load a ply point cloud, print it, and render it


In [5]:
# save the pcd as main.pcd
# Path: data/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/lidar/main.pcd
downpcd = pcd.voxel_down_sample(voxel_size=0.05)
os.makedirs("./home/PJLAB/lipeisen/workspace/bbox-9DoF-annotator/data/0ad2d3a1-79e2-2212-9b99-a96495d9f7fe/lidar", exist_ok=True)
o3d.io.write_point_cloud("/home/PJLAB/lipeisen/workspace/bbox-9DoF-annotator/data/0ad2d3a1-79e2-2212-9b99-a96495d9f7fe/main.pcd", downpcd)

True

In [5]:
mesh = o3d.io.read_triangle_mesh("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/mesh.refined.v2.obj")

In [6]:
o3d.visualization.draw_geometries([mesh])

In [8]:
mesh

geometry::TriangleMesh with 59970 points and 90747 triangles, and (4096, 4096) texture.

In [7]:
texture = np.asarray(mesh.texture)
texture.shape
texture[1,1,]

array([125, 126, 125], dtype=uint8)

In [9]:
with open("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/mesh.refined.0.010000.segs.v2.json") as f:
    mesh_refined = json.load(f)

In [10]:
with open("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/semseg.v2.json") as f:
    semseg = json.load(f)


In [11]:
seg_groups = semseg["segGroups"]

In [12]:
seg_groups[0]

{'objectId': 1,
 'id': 1,
 'partId': 1,
 'index': 0,
 'dominantNormal': [0, -2.220446049250313e-16, 1],
 'obb': {'axesLengths': [6.178954373532301,
   0.5182499173674748,
   3.3831135437636566],
  'centroid': [0.358748431470707, 0.3052196121981847, -1.1691250260369859],
  'normalizedAxes': [0.7100464701652527,
   -0.7041548490524292,
   -1.563537852638917e-16,
   0,
   -2.220446049250313e-16,
   1,
   -0.7041548490524292,
   -0.7100464701652527,
   -1.5766198794625656e-16]},
 'segments': [146,
  608,
  625,
  1089,
  3822,
  3927,
  4147,
  4434,
  4708,
  8811,
  9593,
  10416,
  10715,
  10724,
  10749,
  10785,
  10808,
  10809,
  11041,
  11048,
  12070,
  12690,
  14418,
  16172,
  18618,
  19449,
  20436,
  20985,
  30704,
  31746,
  34609,
  36069,
  36109,
  36138,
  36142,
  36370,
  36382,
  37626,
  40084,
  42684,
  43199,
  43225,
  47194,
  47198,
  47213,
  47226,
  47236,
  47237,
  47241,
  47247,
  47254,
  48246,
  48254,
  48272,
  48359,
  49150,
  51478,
  51484,


In [13]:
def axes2rotation(obb):
    R = np.array(obb["normalizedAxes"]).reshape(3, 3)
    roll = np.arctan2(R[2, 1], R[2, 2])
    pitch = np.arctan2(-R[2, 0], np.sqrt(R[2, 1]**2 + R[2, 2]**2))
    yaw = np.arctan2(R[1, 0], R[0, 0])
    return [roll, pitch, yaw]



def convert_seg_group(seg_group):
    obb = seg_group["obb"]
    main_info = {
        "obj_id": str(seg_group["objectId"]),
        "obj_type": str(seg_group["label"]),
        "psr": {
            "position": {
                "x": obb["centroid"][0],
                "y": obb["centroid"][1],
                "z": obb["centroid"][2]
            },
            "scale": {
                "x": obb["axesLengths"][0],
                "y": obb["axesLengths"][1],
                "z": obb["axesLengths"][2]
            },
            "rotation": {
                "x": axes2rotation(obb)[0],
                "y": axes2rotation(obb)[1],
                "z": axes2rotation(obb)[2]
            }
        }
    }

    return main_info

In [14]:
seg_groups = semseg["segGroups"]
main = []
for seg_group in seg_groups:
    main.append(convert_seg_group(seg_group))

# save to json file
with open('my_sem_seg.json', 'w') as outfile:
    json.dump(main, outfile, indent=2)



In [20]:
seg_groups[1]['obb']['centroid']

[1.811295440998764, -0.8993162065076098, -0.1680325250767365]

In [24]:
def show_obb(obb_idx):
    obb = o3d.geometry.OrientedBoundingBox()
    obb.center = seg_groups[obb_idx]['obb']['centroid']
    # trans the normalized axes to the 3*3 numpy.ndarray
    # 转置归一化的轴到3*3的numpy.ndarray
    obb.R = np.array(seg_groups[obb_idx]['obb']['normalizedAxes']).reshape(3,3).T

    obb.extent = seg_groups[obb_idx]['obb']['axesLengths']
    obb_frame = o3d.geometry.LineSet.create_from_oriented_bounding_box(obb)
    obb_frame.paint_uniform_color([1, 0, 0])  # 设置边框颜色

    # 画出包围盒
    o3d.visualization.draw_geometries([pcd, obb_frame])


In [27]:
show_obb(3)

In [26]:
import open3d as o3d
import json

# 读取点云文件
pcd = o3d.io.read_point_cloud("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/labels.instances.annotated.v2.ply")

# 读取semseg数据
with open("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/semseg.v2.json") as f:
    semseg = json.load(f)
with open("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/sam/000000.json") as f:
    sam = json.load(f)

# 读取mesh数据
mesh = o3d.io.read_triangle_mesh("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/mesh.refined.v2.obj")

# 创建一个空的点云对象
obb_frames = o3d.geometry.LineSet()
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = pcd.points

# 创建一个OBB队列
obb_queue = Queue()
obb_label = Queue()

# 将所有semseg中的OBB参数添加到OBB队列中
'''
for obb_param in semseg['segGroups']:
    obb = o3d.geometry.OrientedBoundingBox()
    obb.center = obb_param["obb"]["centroid"]
    obb.extent = obb_param["obb"]["axesLengths"]
    obb.R = np.array(obb_param["obb"]["normalizedAxes"]).reshape(3, 3).T
    label = obb_param["label"]
    obb_label.put(label)
    obb_queue.put(obb)
'''
for obb_param in sam:
    obb = o3d.geometry.OrientedBoundingBox()
    obb.center = [obb_param["position"]['x'], obb_param["position"]['y'], obb_param["position"]['z']]
    obb.extent = [obb_param["scale"]['x'], obb_param["scale"]['y'], obb_param["scale"]['z']]
    obb.R = np.eye(3)
    obb_queue.put(obb)


# 从OBB队列中获取所有OBB对象并绘制边框
while not obb_queue.empty():
    obb = obb_queue.get()
    obb_frame = o3d.geometry.LineSet.create_from_oriented_bounding_box(obb)
    obb_frame.paint_uniform_color([1, 0, 0])  # 设置边框颜色
    # label = o3d.geometry.LineSet.create_text(obb.center, label, 0.1)
    obb_frames += obb_frame
    # obb_frames += label


# 可视化点云和OBB边框
o3d.visualization.draw_geometries([mesh, obb_frames])

In [76]:
with open("/datax/3rscan/0a4b8ef6-a83a-21f2-8672-dce34dd0d7ca/sequence/_info.txt", 'r') as info_file:
    lines = info_file.readlines()
    depth_intrinsic = lines[-3].split()[-16:-1]

In [86]:
obb_queue = Queue()
for obb_param in sam:
    obb = o3d.geometry.OrientedBoundingBox()
    obb.center = [obb_param["position"]['x'], obb_param["position"]['y'], obb_param["position"]['z']]
    obb.extent = [obb_param["scale"]['x']*100, obb_param["scale"]['y']*100, obb_param["scale"]['z']*100]
    obb.R = np.eye(3)
    obb_queue.put(obb)

In [73]:
semseg['segGroups']

[{'objectId': 1,
  'id': 1,
  'partId': 1,
  'index': 0,
  'dominantNormal': [0, -2.220446049250313e-16, 1],
  'obb': {'axesLengths': [6.178954373532301,
    0.5182499173674748,
    3.3831135437636566],
   'centroid': [0.358748431470707, 0.3052196121981847, -1.1691250260369859],
   'normalizedAxes': [0.7100464701652527,
    -0.7041548490524292,
    -1.563537852638917e-16,
    0,
    -2.220446049250313e-16,
    1,
    -0.7041548490524292,
    -0.7100464701652527,
    -1.5766198794625656e-16]},
  'segments': [146,
   608,
   625,
   1089,
   3822,
   3927,
   4147,
   4434,
   4708,
   8811,
   9593,
   10416,
   10715,
   10724,
   10749,
   10785,
   10808,
   10809,
   11041,
   11048,
   12070,
   12690,
   14418,
   16172,
   18618,
   19449,
   20436,
   20985,
   30704,
   31746,
   34609,
   36069,
   36109,
   36138,
   36142,
   36370,
   36382,
   37626,
   40084,
   42684,
   43199,
   43225,
   47194,
   47198,
   47213,
   47226,
   47236,
   47237,
   47241,
   47247,
   4

In [79]:
sam[1]

{'position': {'x': 0.31378873662873297,
  'y': 0.05417565235236376,
  'z': 0.06822622415019713},
 'rotation': {'x': 0.0, 'y': 0.0, 'z': 0.0},
 'scale': {'x': 0.014771496150573216,
  'y': 0.007815902707922151,
  'z': 0.018101705835812068}}

In [9]:
import os
os.chdir('/datax/3rscan/')
folder_names = os.listdir()

In [22]:
folder_names[7]

'74a2f6e3-b052-2688-81cf-5de5a1b6948e'

In [24]:
lidar_dir = os.path.join("/datax/3rscan",folder_names[9], "labels.instances.annotated.v2.ply")
pcd = o3d.io.read_point_cloud(lidar_dir)
o3d.visualization.draw_geometries([pcd]) 

In [26]:
print(folder_names[1])
print(folder_names[9])

0cac7584-8d6f-2d13-8df8-c05e4307b418
c7895f2f-339c-2d13-8388-28e95af958de


In [ ]:
def get_pcd(data_dir, data_init_dir, folder_name, color_name):
    info_path = os.path.join(data_dir, folder_name, "sequence", "_info.txt")
    with open(info_path, 'r') as info_file:
        lines = info_file.readlines()
        num_frames = int(lines[-1].split()[-1])
        depth_intrinsic = float(lines[-3].split()[-16:-1])

    pose = os.path.join(rgb_path, scene_name, color_name + '.pose.txt')
    depth = os.path.join(rgb_path, scene_name, color_name + '.depth.pgm')
    color = os.path.join(rgb_path, scene_name, color_name + '.color.jpg')
    depth_img = cv2.imread(depth, cv2.IMREAD_UNCHANGED)
    mask = (depth_img != 0)
    color_image = cv2.imread(color)
    color_image = cv2.resize(color_image, (640, 480))

    seg_2d_path = os.path.join(data_init_dir, folder_name, "sam_2dmask", color_name + '.jpg')
    seg_2d_img = cv2.imread(seg_2d_path, cv2.IMREAD_UNCHANGED)

    group_path = os.path.join(data_init_dir, folder_name, "sam_2dmask", color_name + '.json')
    with open(group_path) as f:
        group_ids = json.load(f)
    group_ids = np.array(group_ids)
    ori_group_ids = group_ids

    novel_group_ids = get_novel(seg_2d_img, group_ids)

    color_image = np.reshape(color_image[mask], [-1,3])
    group_ids = group_ids[mask]
    colors = np.zeros_like(color_image)
    colors[:,0] = color_image[:,2]
    colors[:,1] = color_image[:,1]
    colors[:,2] = color_image[:,0]

    pose = np.loadtxt(pose)

    depth_shift = 1000.0
    x,y = np.meshgrid(np.linspace(0,depth_img.shape[1]-1,depth_img.shape[1]), np.linspace(0,depth_img.shape[0]-1,depth_img.shape[0]))
    uv_depth = np.zeros((depth_img.shape[0], depth_img.shape[1], 3))
    uv_depth[:,:,0] = x
    uv_depth[:,:,1] = y
    uv_depth[:,:,2] = depth_img/depth_shift
    uv_depth = np.reshape(uv_depth, [-1,3])
    uv_depth = uv_depth[np.where(uv_depth[:,2]!=0),:].squeeze()

    fx = depth_intrinsic[0]
    fy = depth_intrinsic[5]
    cx = depth_intrinsic[2]
    cy = depth_intrinsic[6]
    bx = depth_intrinsic[3]
    by = depth_intrinsic[7]
    n = uv_depth.shape[0]
    points = np.ones((n,4))
    X = (uv_depth[:,0]-cx)*uv_depth[:,2]/fx + bx
    Y = (uv_depth[:,1]-cy)*uv_depth[:,2]/fy + by
    points[:,0] = X
    points[:,1] = Y
    points[:,2] = uv_depth[:,2]
    points_world = np.dot(points, np.transpose(pose))
    return points_world, group_ids, novel_group_ids, ori_group_ids, False


In [ ]:
def get_box(points, groups, novel_group_ids):
    matrix = np.eye(4)
    points_aligned = np.dot(points, np.transpose(matrix))
    obj_ids = np.unique(groups)
    res = []
    rev_mapping = []
    for i in range(obj_ids.shape[0]):
        obj = obj_ids[i]
        if obj not in novel_group_ids:
            continue
        if obj < 0:
            continue
        rev_mapping.append(obj)
        mask = (groups == obj)
        box_point = points_aligned[mask]
        
        xmin = np.min(box_point[:,0])
        xmax = np.max(box_point[:,0])
        ymin = np.min(box_point[:,1])
        ymax = np.max(box_point[:,1])
        zmin = np.min(box_point[:,2])
        zmax = np.max(box_point[:,2])
        xmid = (xmin + xmax) / 2.0
        ymid = (ymin + ymax) / 2.0
        zmid = (zmin + zmax) / 2.0
        xscale = xmax - xmin
        yscale = ymax - ymin
        zscale = zmax - zmin
        res.append({"position": {"x" : xmid, "y":ymid, "z":zmid},
                    "rotation": {"x" : 0.0, "y": 0.0, "z": 0.0},
                    "scale": {"x" : xscale, "y" : yscale, "z":zscale}
                    })
    return res, rev_mapping

In [58]:
def get_novel(seg, seg2):
    # print('-----------------------------------------')
    # print(time.time())
    seg1 = cv2.resize(seg, (seg2.shape[1], seg2.shape[0]), interpolation=cv2.INTER_NEAREST).astype(seg2.dtype)
    iou_thr = 0.5
    # assert seg1.shape == seg2.shape
    seg1 = seg1 - 1 # 0-based ids
    a = np.unique(seg1)
    b = np.unique(seg2)
    mina = int(np.min(a))
    minb = int(np.min(b))
    count_a = np.zeros_like(a)
    count_b = np.zeros_like(b)
    count = np.zeros((a.shape[0], b.shape[0]), dtype=int)
    indexa = np.zeros((int(np.max(a) - mina + 1),), dtype=int)
    indexb = np.zeros((int(np.max(b) - minb + 1),), dtype=int)
    for i in range(a.shape[0]):
        indexa[int(a[i])- mina] = i
    for i in range(b.shape[0]):
        indexb[int(b[i]) - minb] = i
    # print(time.time())
    for i in range(seg1.shape[0]):
        for j in range(seg1.shape[1]):
            idxa = indexa[seg1[i][j] - mina]
            idxb = indexb[seg2[i][j] - minb]
            count_a[idxa] += 1
            count_b[idxb] += 1
            count[idxa][idxb] += 1
    
    # print(time.time())
    # print(a)
    # print(b)
    res = []
    for j in range(len(b)):
        if int(b[j]) != -1:
            bo = False
            for i in range(len(a)):
                if int(a[i]) != -1:
                    iou = count[i][j] / (count_a[i] + count_b[j] - count[i][j])
                    if iou > iou_thr:
                        # print(a[i], b[j], iou)
                        bo = True
                        break
            if not bo:
                res.append(b[j])
    # import pdb
    # pdb.set_trace()
    # print(time.time())
    # print('--------------------------------')
    return res

In [30]:
script_dir = "/home/PJLAB/lipeisen/workspace/3Rscan"

data_init_path = os.path.join(script_dir, "data_init")

data_path = os.path.join(script_dir, "data")

folder_name = "0ad2d3a1-79e2-2212-9b99-a96495d9f7fe"

color_name = "frame-000000"

In [36]:
seg_2d_path = os.path.join(data_init_path, folder_name, "sam_2dmask", color_name + '.color.jpg') # TODO: delete '.color'
seg_2d_img = cv2.imread(seg_2d_path, cv2.IMREAD_UNCHANGED)

group_path = os.path.join(data_init_path, folder_name, "sam_2dmask", color_name + '.json')
with open(group_path) as f:
    group_ids = json.load(f)
group_ids = np.array(group_ids)
ori_group_ids = group_ids

In [37]:
seg_2d_path = os.path.join(data_init_path, folder_name, "sam_2dmask", color_name + '.color.jpg') # TODO: delete '.color'
seg_2d_img = cv2.imread(seg_2d_path, cv2.IMREAD_UNCHANGED)

group_path = os.path.join(data_init_path, folder_name, "sam_2dmask", color_name + '.json')
with open(group_path) as f:
    group_ids = json.load(f)
group_ids = np.array(group_ids)
ori_group_ids = group_ids

#novel_group_ids = get_novel(seg_2d_img, group_ids)

In [42]:
seg_2d_img.shape

(540, 960, 3)

In [61]:
np.unique(group_ids)

array([-1.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25.,
       26., 27., 28., 29., 30., 31., 32.])

In [60]:
np.unique(seg_2d_img)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [44]:
seg_2d_img - 1

array([[[ 20, 160, 129],
        [ 20, 163, 131],
        [ 19, 162, 129],
        ...,
        [188,  59,  32],
        [188,  60,  31],
        [188,  60,  31]],

       [[ 22, 162, 131],
        [ 19, 162, 130],
        [ 18, 161, 128],
        ...,
        [163,  18,   2],
        [163,  18,   2],
        [163,  18,   2]],

       [[ 21, 161, 130],
        [ 20, 163, 131],
        [ 19, 162, 130],
        ...,
        [184,   7,  16],
        [186,   7,  16],
        [186,   8,  15]],

       ...,

       [[ 87,  34,  74],
        [ 86,  34,  74],
        [ 87,  34,  74],
        ...,
        [149,  68, 195],
        [149,  68, 195],
        [149,  68, 195]],

       [[ 12, 255,   8],
        [ 11, 255,   8],
        [ 12, 255,   8],
        ...,
        [ 12, 255,  27],
        [ 12, 255,  27],
        [ 12, 255,  27]],

       [[  0, 255,   1],
        [  0, 255,   1],
        [  0, 255,   1],
        ...,
        [  1, 255, 255],
        [  1, 255, 255],
        [  1, 255, 255]]

In [43]:
group_ids.shape

(540, 960)

In [59]:
novel_group_ids = get_novel(seg_2d_img, group_ids)

TypeError: only size-1 arrays can be converted to Python scalars